# Lie Group SO(3) Applied to Quadcopter
This notebook will go through the dynamics of a quadrotor when using the SO(3) Lie Group attitude representation as well as how an MPC controller can be designed.

## Nomenclature

$\textbf{p}_{b/i}^i$: position of quadrotor's body frame with respect to the inertial frame, expressed in the inertial frame

$\textbf{v}_{b/i}^b$: velocity of body frame with respect to inertial frame, expressed in the body frame

$R_b^v$: rotation matrix describing the attitude of the body frame, expressed in the vehicle frame

$\textbf{$\omega$}_{b/i}^b$: angular velocity of the body frame with respect to the inertial frame, expressed in the body frame

$\textbf{f}^b$: total force produced from all 4 motors expressed in body frame

$\textbf{$\tau$}^b$: torque about 3 axes of body frame expressed in body frame

[.]$^\wedge$: skew-symmetric operator - maps from $R^3$ to $so(3)$ and functions as a cross product operator

[.]$^\vee$: vee operator - maps from $so(3)$ to $R^3$

## Dynamics

$\dot{\textbf{p}}_{b/i}^i = R_b^v \textbf{v}_{b/i}^b$

$\dot{\textbf{v}}_{b/i}^b = [\textbf{v}_{b/i}^b]^\wedge\textbf{$\omega$}_{b/i}^b -\frac{\textbf{f}^b}{m} + R_v^b \textbf{g}^v - \frac{\mu}{m} \textbf{v}_{b/i}^b$

$\dot{R}_b^v = R_b^v [\textbf{$\omega$}_{b/i}^b]^\wedge$

$\dot{\textbf{$\omega$}}_{b/i}^b = J^{-1} ([-\textbf{$\omega$}_{b/i}^b]^\wedge J \textbf{$\omega$}_{b/i}^b + \textbf{$\tau$}^b)$

## Control Design
In order to properly use the log function to vectorize the SO(3) manifold, we need to use error state dynamics. The derivation below shows how we will be using $R_r$, which is the reference rotation matrix. Note that the disriptors on each variable indicating frame of reference are dropped from here on out for readability, but the variables are all used as described above.

$\tilde{\Phi} = [log(R_r^T R)]^\vee$

$exp([\tilde{\Phi}]^\wedge) = R_r^T R$

$exp([\tilde{\Phi}]^\wedge)R^T = R_r^T$

$R_r = R exp([\tilde{\Phi}]^\wedge)^T$

$R_r = R exp(([\tilde{\Phi}]^\wedge)^T)$

$R_r = R exp(-[\tilde{\Phi}]^\wedge)$

Knowing that $exp(X) = I + X + \frac{X^2}{2!}+...$ and that $[\tilde{\Phi}]^\wedge$ is an error term (meaning that it should be small, especially when multiplied by itself),

$R_r \approx R(I-[\tilde{\Phi}]^\wedge) = R-R[\tilde{\Phi}]^\wedge$

This will be used a few times to replace $R_r$ later on.

### Derivation of Error State Dynamics

$\dot{\tilde{p}} = \dot{p} - \dot{p}_r$ 

$\ \ \ = Rv - R_r v_r$

$\ \ \ = Rv - R \exp{(-[\tilde{\Phi}]^\wedge)} (v-\tilde{v})$

$\ \ \ \approx Rv - R (I-[\tilde{\Phi}]^\wedge) (v-\tilde{v})$

$\ \ \ = Rv - (R-R[\tilde{\Phi}]^\wedge) (v-\tilde{v})$

$\ \ \ = Rv - Rv + R\tilde{v} + R[\tilde{\Phi}]^\wedge v -R[\tilde{\Phi}]^\wedge\tilde{v}\ \ \ $ any time 2 tilde variables are multiplied, I assume they are 0

$\ \ \ \approx R\tilde{v} + R[\tilde{\Phi}]^\wedge v$

$\ \ \ = R\tilde{v} - R[v]^\wedge\tilde{\Phi}$

$\dot{\tilde{\Phi}} = \dot{\Phi} - \dot{\Phi}_r$

$\ \ \ = \omega - \omega_r$

$\ \ \ = \tilde{\omega}$

(The above derivation came from James)

$\dot{\tilde{v}} = \dot{v} - \dot{v}_r$

$\ \ \ = ([v]^\wedge\omega - \frac{f}{m} +R^Tg-\frac{\mu}{m} v) - ([v_r]^\wedge\omega_r - \frac{f_r}{m} +R_r^Tg - \frac{\mu}{m} v_r)$

$\ \ \ = ([v]^\wedge\omega - \frac{f}{m} +R^Tg-\frac{\mu}{m} v) - ([v-\tilde{v}]^\wedge(\omega-\tilde{\omega}) - \frac{f-\tilde{f}}{m} +(Rexp(-[\tilde{\Phi}]^\wedge))^Tg -\frac{\mu}{m} (v-\tilde{v}))$

$\ \ \ = ([v]^\wedge\omega - \frac{f}{m} +R^Tg-\frac{\mu}{m} v) - ([v]^\wedge\omega-[v]^\wedge\tilde{\omega} +[\omega]^\wedge\tilde{v}+[\tilde{v}]^\wedge\tilde{\omega} - \frac{f-\tilde{f}}{m} +exp([\tilde{\Phi}]^\wedge)R^Tg -\frac{\mu}{m} v +\frac{\mu}{m}\tilde{v})$

$\ \ \ \approx ([v]^\wedge\omega - \frac{f}{m} +R^Tg-\frac{\mu}{m} v) - ([v]^\wedge\omega-[v]^\wedge\tilde{\omega} +[\omega]^\wedge\tilde{v}+[\tilde{v}]^\wedge\tilde{\omega} - \frac{f-\tilde{f}}{m} +(I-[\tilde{\Phi}]^\wedge)R^Tg -\frac{\mu}{m} v +\frac{\mu}{m}\tilde{v}))$

$\ \ \ = ([v]^\wedge\omega - \frac{f}{m} +R^Tg-\frac{\mu}{m} v) - ([v]^\wedge\omega-[v]^\wedge\tilde{\omega}+[\omega]^\wedge\tilde{v}+[\tilde{v}]^\wedge\tilde{\omega} - \frac{f-\tilde{f}}{m} +R^Tg-[\tilde{\Phi}]^\wedge R^Tg -\frac{\mu}{m} v +\frac{\mu}{m}\tilde{v}))$

$\ \ \ = [v]^\wedge\tilde{\omega}-[\omega]^\wedge\tilde{v}-[\tilde{v}]^\wedge\tilde{\omega} - \frac{\tilde{f}}{m}-[\tilde{\Phi}]^\wedge R^Tg -\frac{\mu}{m}\tilde{v})$

$\ \ \ \approx [v]^\wedge\tilde{\omega}-[\omega]^\wedge\tilde{v}+[R^Tg]^\wedge\tilde{\Phi} - \frac{\tilde{f}}{m} - \frac{\mu}{m} \tilde{v}$

$\dot{\tilde\omega} = \dot{\omega} - \dot{\omega}_r$

$\ \ \ = (-J^{-1}[\omega]^\wedge J\omega + J^{-1}\tau) - (-J^{-1}[\omega_r]^\wedge J\omega_r + J^{-1}\tau_r)$

$\ \ \ = (-J^{-1}[\omega]^\wedge J\omega + J^{-1}\tau) - (-J^{-1}[\omega-\tilde{\omega}]^\wedge J(\omega-\tilde{\omega}) + J^{-1}(\tau-\tilde{\tau}))$

$\ \ \ = J^{-1}\tilde{\tau} -J^{-1}[\omega]^\wedge J\omega +J^{-1}[\omega-\tilde{\omega}]^\wedge J(\omega-\tilde{\omega})$

$\ \ \ = J^{-1}\tilde{\tau} -J^{-1}[\omega]^\wedge J\omega +J^{-1}[\omega-\tilde{\omega}]^\wedge J\omega-J^{-1}[\omega-\tilde{\omega}]^\wedge J\tilde{\omega}$

$\ \ \ = J^{-1}\tilde{\tau} -J^{-1}[\omega]^\wedge J\omega +J^{-1}[\omega]^\wedge J\omega-J^{-1}[\tilde{\omega}]^\wedge J\omega  -J^{-1}[\omega]^\wedge J\tilde{\omega} +J^{-1}[\tilde{\omega}]^\wedge J\tilde{\omega}$

$\ \ \ = J^{-1}\tilde{\tau} -J^{-1}[\tilde{\omega}]^\wedge J\omega  -J^{-1}[\omega]^\wedge J\tilde{\omega} +J^{-1}[\tilde{\omega}]^\wedge J\tilde{\omega}$

$\ \ \ = J^{-1}\tilde{\tau} +J^{-1}[J\omega]^\wedge\tilde{\omega}  -J^{-1}[\omega]^\wedge J\tilde{\omega} +J^{-1}[\tilde{\omega}]^\wedge J\tilde{\omega}$

$\ \ \ \approx J^{-1}\tilde{\tau} +J^{-1}([J\omega]^\wedge-[\omega]^\wedge J)\tilde{\omega}$

### Summary of Error State Dynamics
These are approximations for the dynamics we will use to derive state-space matrices. We will be using a state-space model for the MPC controller. Remember that $\dot{\tilde{x}} = A\tilde{x}+B\tilde{u}$

$\dot{\tilde{p}} \approx R\tilde{v} - R[v]^\wedge\tilde{\Phi}$

$\dot{\tilde{\Phi}} \approx \tilde{\omega}$

$\dot{\tilde{v}} \approx [v]^\wedge\tilde{\omega}-[\omega]^\wedge\tilde{v}+[R^Tg]^\wedge\tilde{\Phi} - \frac{\tilde{f}}{m} - \frac{\mu}{m} \tilde{v}$

$\dot{\tilde\omega} \approx J^{-1}\tilde{\tau} +J^{-1}([J\omega]^\wedge-[\omega]^\wedge J)\tilde{\omega}$

### Jacobian Linearization
In order to find the A and B matrices, we will use Jacobian linearization on the equations for error state dynamics. We will partially linearize about equilibrium as will be explained. We will assume that the equilibrium for the attitude of the quadcopter is its current attitude: $R_e = R$. All of the other equilibrium values will remain at true equilibrium: $\tau_e=v_e=\omega_e=p_e=[0,0,0]^T, \ \ f_e = mg$. 

$\tilde{x}=[\tilde{p},\tilde{\Phi},\tilde{v},\tilde{\omega}]^T,\ \ \ \ \tilde{u}=[\tilde{f},\tilde{\tau}]^T$

$A = \frac{\partial\dot{\tilde{x}}}{\partial\tilde{x}}|_e,\ \ \ \ B=\frac{\partial\dot{\tilde{x}}}{\partial\tilde{u}}|_e$

$A=\begin{pmatrix} 0 & -R_e[v_e]^\wedge & R_e & 0 \\ 0 & 0 & 0 & I \\ 0 & [R_e^Tg]^\wedge & -\frac{\mu}{m} I-[\omega_e]^\wedge & [v_e]^\wedge \\ 0 & 0 & 0 & [J\omega_e]^\wedge-[\omega_e]^\wedge J \end{pmatrix} = \begin{pmatrix} 0 & 0 & R & 0 \\ 0 & 0 & 0 & I \\ 0 & [R^Tg]^\wedge & -\frac{\mu}{m} I & 0 \\ 0 & 0 & 0 & 0 \end{pmatrix}$

$B=\begin{pmatrix} 0_{8x1} & 0_{8x3} \\ -\frac{1}{m} & 0_{1x3} \\ 0_{3x1} & J^{-1} \end{pmatrix}$

If you want to use motor signals as inputs $(\tilde{u}=[\delta_f,\delta_r,\delta_b,\delta_l]^T)$ with the assumption that $F_*=k_1\delta_*$ and $\tau_*=k_2\delta_*$, then

$B=\begin{pmatrix} 0_{8x1} & 0_{8x3} \\ -\frac{1}{m} & 0_{1x3} \\ 0_{3x1} & J^{-1} \end{pmatrix} \begin{pmatrix} -\frac{k_1}{m} & -\frac{k_1}{m}&-\frac{k_1}{m}&-\frac{k_1}{m} \\ 0 & -\frac{d k_1}{J_x}&0&\frac{d k_1}{J_x} \\ \frac{d k_1}{J_y} & 0 & -\frac{d k_1}{J_y}&0 \\ -\frac{k_2}{J_z} & \frac{k_2}{J_z}&-\frac{k_2}{J_z}&\frac{k_2}{J_z} \end{pmatrix}$

### MPC Design
The time horizon and the discretization time could be chosen in a different way, but I used a time horizon of 10 steps with the size of the step (discretization time) being the settling time of the system divided by the time horizon. This means my MPC controller will take 10 equally spaced steps to predict 4 seconds into the future. Theoretically, better performance would come by increasing the time horizon and decreasing the discretization time; however, this becomes much more computationally expensive. Through some experimentation, I found that a time horizon greater than 10 usually doesn't make much of a difference. This could be due to the fact that MPC only takes the 1st of the predicted steps and then recalculates everything all over again.

$t_{settle} = 4s,\ \ \ N=10$

$T_s = \frac{t_{settle}}{N} = 0.4s \ \ \ \  $ where $T_s$ is the discretization time (step size) for the A and B matrices

#### Cost Function

$J = \sum_{k=0}^N \{(x[k]-x_{ss})^T W_x (x[k]-x_{ss})+(u[k]-u_{ss})^T W_u (u[k]-u_{ss})\}$

#### Constraints

$x[k+1] == A(x[k]-x_e) + B(u[k]-u_e)\ \ \ $ this says that the system's dynamics must be true

$u_{min} <= u[k] <= u_{max}\ \ \ $ this allows MPC to know the input saturation limits

One thing to note is that A needs to be updated at the beginning of each MPC control calculation. Although A is recalculated at every time step, it is a static value in MPC's eyes when predicting into the future. This could potentially be changed where MPC updates A for each step into the future, but that would be more computationally expensive.